In [41]:

from Crypto.Util.number import *
from secrets import randbits


def rc4(key, data):
    S = [randbits(8) for _ in range(256)]
    j = 0

    for idx in range(len(key)):
        i = idx & 0xff
        j = (j + S[i] + key[i % len(key)]) % 256
        S[i] ^= S[j]
        S[j] ^= S[i]
        S[i] ^= S[j]

    i = 0
    j = 0
    output = []
    for byte in data:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i] ^= S[j]
        S[j] ^= S[i]
        S[i] ^= S[j]
        K = S[(S[i] + S[j]) % 256]
        output.append(byte ^ K)
    return bytes(output)

flag = b"ictf{this-is-my-s3cret-flag-laalal-b1tche}"
ciphertext = rc4(b"",flag)

In [ ]:
 S = [randbits(8) for _ in range(256)]
key = b"\x00"
for idx in range(len(key)):
    i = idx & 0xff
    j = (j + S[i] + key[i % len(key)]) % 256
    S[i] ^= S[j]
    S[j] ^= S[i]
    S[i] ^= S[j]


# Sboxes

## Design

```python
 S = [randbits(8) for _ in range(256)]
```

are in original form


```python
 S = list(range(256))
```

because they permutate the data. Therefore, this could actually mess up the data pretty bad.

## Init

The sboxes are random and the code a little messed up

```python
    for idx in range(len(key)):
        i = idx & 0xff
        j = (j + S[i] + key[i % len(key)]) % 256
        S[i] ^= S[j]
        S[j] ^= S[i]
        S[i] ^= S[j]
```

equals to

```python
    for idx in range(len(key)):
        i = idx & 0xff # the keylenth is therefore 256, however
        j = (j + S[i] + key[i % len(key)]) % 256
        S[i],S[j] = S[j],S[i]  
```

In [14]:
S = list(range(256))
key = [1]*(1)
j=0
for idx in range(len(key)):
    i = idx & 0xff # the keylenth is therefore 256, however
    j = (j + S[i] + key[i % len(key)]) % 256
    S[i],S[j] = S[j],S[i]
print(S)

[1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

We can however note, that no key length is enforced, so we can also put in an empty key. Furthermore, the flag starts with `"ictf{"`, such that we have some plaintext.

# Encrpytion

```python
    i = 0
    j = 0
    for byte in data:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i] ^= S[j]
        S[j] ^= S[i]
        S[i] ^= S[j]
        K = S[(S[i] + S[j]) % 256]
        output.append(byte ^ K)
    return bytes(output)
```

equals

```python
    i = 0
    j = 0
    for byte in data:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i],S[j] = S[j],S[i]
        K = S[(S[i] + S[j]) % 256]
        output.append(byte ^ K)
    return bytes(output)
```

is independent of the key, so only about the boxes (apparently, they are not sboxes)

# Let's try z3

In [20]:
"""
❯ nc 8.138.23.35 8000
🔑 
🔒 b'\\#*b\xdd\xee\xadt\xa6@\x8dj9C\xfe\xb6\xe7\xf9k\x8c:8*H"ms\x9f\xd8\x18\xfc\x0c~\x11>F\x9bN\xcbTF\r'

❯ nc 8.138.23.35 8000
🔑 1
🔒 b'\xf2\xfa\xf9\xb7c\xfa\xaf%\xf1\xed\xe5\x97v\xbd\x90\xce\xfaK\x1e\x11\x92\xd1\x9b\n\xc7\xf62\xf88\x96\x17C.d\xab\xcc\xfa\xf0\x8c\x97\x9d\x1e'
""" 
ciphertext = b'\x98\xdb\x05\x85\xfex\xd2O\xec\xd8\x14i\x15\xcajz\xef\x9e\xdc$\x1c\x1f\\r\xdd\xf2\xe7\x97i\x8bP\xf8\\\xd1\x0e\xd0\x1b\x90\x8e1\xa8\x0c'
assert len(ciphertext) == 42

In [44]:
from z3 import *

# Define a symbolic array for S (8-bit index -> 8-bit value)
S = Array('S', BitVecSort(8), BitVecSort(8))

# Initialize S with symbolic values
for i in range(256):
    S = Store(S, BitVecVal(i, 8), BitVec(f"S_{i}", 8))

# Define unknown input data (symbolic bytes)
data = [BitVec(f"data_{i}", 8) for i in range(42)]

i = BitVecVal(0, 8)
j = BitVecVal(0, 8)
computed_output = []

for idx in range(42):
    i = (i + 1) & 0xFF
    j = (j + S[i]) & 0xFF

    si = S[i]
    S = Store(S, i, S[j])
    S = Store(S, j, si)

    # Compute keystream byte
    K = S[(S[i] + S[j]) & 0xFF] 
    # Compute decrypted byte
    computed_output.append(data[idx] ^ K)

In [45]:
import string
allowed = set((string.ascii_letters+string.digits+string.punctuation+" ").encode())
allowed.remove(ord("{"))
allowed.remove(ord("}"))

In [47]:
set_param('parallel.enable', True)
s = Solver()
s.set("threads", 4)
output = [BitVecVal(byte, 8) for byte in ciphertext] 
for idx in range(42):
    s.add(computed_output[idx] == output[idx])
for i,known in enumerate(b"ictf{"):
    s.add(data[i] == BitVecVal(known,8))
s.add(data[-1] == BitVecVal(ord("}"),8))

while True:
    print(s.check())
    
    model = s.model()
    recovered_data = bytes([model[data[i]].as_long() for i in range(42)])
    print(recovered_data)
    
    if not set(recovered_data).issubset(allowed):
        for idx in range(5,42-1):
            s.add(data[idx] != BitVecVal(recovered_data[idx],8))

sat
b'ictf{&I\xf6\xcc\xf4\xd4\x82\x84\x00\x06\x89\x00i\xb5FB\x82i\x82\xd2\xd5:\x1a\x12\x88\xf9\x81\x00\x00\x08\x1e\xec\xf4\xea\x00\x81}'
unknown


Z3Exception: model is not available

# Intended easy solution

Yeah, there is an obvious bug

```python
        S[i] ^= S[j]
        S[j] ^= S[i]
        S[i] ^= S[j]
```

ONLY WORK IF $i\neq j$!!! Otherwise, it is zero, which is perfect for as the xor at the end happens with a zero value :)

What is a good key? What is a good key length?

In [71]:
from secrets import randbits
S = [randbits(8) for _ in range(256)]
key = bytes([1]*350_000)
j = 0
for idx in range(len(key)):
    i = idx & 0xff
    j = (j + S[i] + key[i % len(key)]) % 256
    S[i] ^= S[j]
    S[j] ^= S[i]
    S[i] ^= S[j]

S.count(0)

256

```sh
❯ payload=$(head -c 350000 /dev/zero | tr "\0" "\1") && echo -e "$payload\n" | nc 155.248.210.243 42138
🔑 🔒 b'ictf{why_us1ng_tr1pIe_x0r_1n_Crypt0syst3m}'
```